Aufbereitung der Daten aus dem Beispiel Microservices Kubernetes
====================================

![](https://cdn-images-1.medium.com/max/1600/0*o0zVW2O6Rv-LI5Mu.png)

**Collaborative Filtering**: Amazon verwendet eine Technik des maschinellen Lernens Collaborative Filtering genannt (gemeinhin als Empfehlung genannt). So empfiehlt Amazon Nutzern Produkte auf der Grundlage ihrer Geschichte und Ähnlichkeit mit anderen Benutzern.

Dieses Notebook simuliert in einer Vereinfachten Version Produkt Empfehlungen nach.

**Links**:
* [Collaborative Filtering in Python](https://medium.com/@tomar.ankur287/user-user-collaborative-filtering-recommender-system-51f568489727)

Verbindung mit der Datenbank erstellen

In [22]:
%defaultDatasource jdbc:postgresql://postgres.ms-kubernetes/dborder?user=dbuser&password=dbpass

Ausgabe der Kundendaten

In [23]:
select * from customer

Ausgabe der Produktdaten

In [24]:
select * from item

Welcher Kunde hat was bestellt, in der Reihenfolge des Einganges


In [25]:
select  t.*, c.name, ol.item_id, ol.f_count from order_line as ol
    inner join ordertable_order_line as otol
       on  otol.order_line_id = ol.id 
    inner join ordertable as t
       on otol.order_id = t.id 
    inner join customer as c
       on t.customer_id = c.id       
    order by t.id, ol.item_id

Welche Produkte wurden in welcher Menge und Preis bestellt

In [26]:
select  t.customer_id, ol.item_id, i.name, ol.f_count, i.price, ol.f_count * i.price as "amount" from order_line as ol
    inner join ordertable_order_line as otol
       on  otol.order_line_id = ol.id 
    inner join ordertable as t
       on otol.order_id = t.id
    inner join customer as c
       on t.customer_id = c.id
    inner  join item as i
        on ol.item_id = i.id
    order by t.customer_id, ol.item_id  

Aufbereiten der Variablen 

* ${plot} - Kunden und die jeweils bestellen Waren

* ${items} - Bestellte Produkte und deren Anzahl über alle Bestellungen

In [27]:
select  t.customer_id, ol.item_id, i.name, count(ol.item_id) into ${plot} from order_line as ol
    inner join ordertable_order_line as otol
       on  otol.order_line_id = ol.id 
    inner join ordertable as t
       on otol.order_id = t.id
    inner join customer as c
       on t.customer_id = c.id
    inner  join item as i
        on ol.item_id = i.id
    group by t.customer_id, ol.item_id, i.name
    order by t.customer_id, ol.item_id   
    ;
    
select max(i.name), count(ol.f_count) into ${items} from order_line as ol
    inner  join item as i
        on ol.item_id = i.id
        group by i.name
        order by i.name    

Aufbereitung der Daten für Antwort auf die Frage "Kunden welche Produkte X bestellen, bestellten auch Y)

* Verdichtung der Produkte pro Kunde

* Zusammenfassung, Produkt X mit Produkt Y bestellt


In [28]:
%%javascript

var customers = new Map();

// Bestellte Artikel pro Kunde aufbereiten (Warenkorb)
for ( var i = 0; i < beakerx.plot.values.length; i++ )
{
    var c = customers.get( beakerx.plot.values[i][0] );
    if ( c == undefined )
    {
        c = new Map();
        customers.set( beakerx.plot.values[i][0], c )
    }
    var count = c.get( beakerx.plot.values[i][2] )
    if ( count == undefined )
    {
        count = 0;
        c.set( beakerx.plot.values[i][2], 0 );
    }
    var total = count + beakerx.plot.values[i][3];
    c.set( beakerx.plot.values[i][2], total );
}

console.log( customers ) ;
console.log( beakerx.items.values );

// Artikel welche vom Kunden gemeinsam gekauft wurden
var hits = new Map();

for ( var i = 0; i < beakerx.items.values.length; i++ )
{
    for( var [ c, items ] of customers.entries() )
    {
        // Artikel wurde von Kunde gekauft?
        if ( items.get( beakerx.items.values[i] [0] ) != undefined )
        {
            var others = hits.get( beakerx.items.values[i] [0] );
            if ( others == undefined )
            {
                var others = new Map();
                hits.set( beakerx.items.values[i] [0], others );
            }
            var others = new Map( [ ...items, ...others ] );
            others.delete( beakerx.items.values[i] [0] );
            hits.set( beakerx.items.values[i] [0], others );
            console.log( beakerx.items.values[i] [0] + " " + items );
        }
    }
}
console.log( hits );
